In [96]:
import json
import torch, torch.nn as nn
from torch.autograd import Variable
import random


with open("bnc_spoken_output_flattened.json") as infile:
	input_data = json.load(infile)

seq_len = len(text)
batch_size = 1
embedding_size = 1
hidden_size = 1

vocab = set()

for sentance in input_data[0:10]:
    vocab = vocab.union(sentance)

vocab = list(vocab)
vocab_size = len(vocab)
output_size = vocab_size
word_ids = {word: id_ for id_, word in enumerate(vocab)}
ids_word = {id_: word for word, id_ in word_ids.items()}

text = input_data[random.randint(0, 10 - 1)]
indices = [word_ids[word] for word in text]
input_val = torch.eye(vocab_size)[indices].unsqueeze(1)

bi_rnn = torch.nn.RNN(
    input_size=vocab_size, hidden_size=hidden_size, num_layers=1, batch_first=False, bidirectional=True)

bi_output, bi_hidden = bi_rnn(input_val)

# stagger
forward_output, backward_output = bi_output[:-2, :, :hidden_size], bi_output[2:, :, hidden_size:]
staggered_output = torch.cat((forward_output, backward_output), dim=-1)

linear = nn.Linear(hidden_size * 2, output_size)

# only predict on words
labels = torch.LongTensor(indices[1:-1])

# for language models, use cross-entropy :)
loss = nn.CrossEntropyLoss()
output = loss(linear(staggered_output.squeeze(1)), labels)

print(output)

tensor(5.1633, grad_fn=<NllLossBackward0>)


KeyError: 'does'